In [2]:
! pip install spark-nlp

     |████████████████████████████████| 144 kB 5.3 MB/s 


In [3]:
! pip install pyspark

     |████████████████████████████████| 281.4 MB 28 kB/s 
     |████████████████████████████████| 198 kB 57.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=bde06f4ad96cf1479340acc4e0674837f42e0d0fb67d9ed06d9f3b50d80c6c85
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
import sparknlp
import pandas as pd
import numpy as np

Load data

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
%cd /content/gdrive/My Drive/Upwork/April2022/Kishore

/content/gdrive/My Drive/Upwork/April2022/Kishore


In [344]:
data=pd.read_csv('reviews.tsv',on_bad_lines='skip',sep='\t',nrows=5000)

In [345]:
data.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,9970739,R8EWA1OFT84NX,B00GSP5D94,329991347,Summer Infant SwaddleMe Adjustable Infant Wrap...,Baby,5,0,0,N,Y,Great swaddled blankets,Loved these swaddle blankets and so did my dau...,2015-08-31
1,US,23538442,R2JWY4YRQD4FOP,B00YYDDZGU,646108902,Pacifier Clip Girl (3 Pack) Ziggy Baby 2-Sided...,Baby,5,0,0,N,N,Too cute and really nice,These are adorable pacifier clips. SavvyBaby h...,2015-08-31
2,US,8273344,RL5ESX231LZ0B,B00BUBNZC8,642922361,Udder Covers - Breast Feeding Nursing Cover,Baby,5,0,0,N,Y,Five Stars,Great gift,2015-08-31
3,US,24557753,RRMS9ZWJ2KD08,B00AWLZFTS,494272733,Gerber Graduates Fun Pack Utensils,Baby,5,0,0,N,Y,Cute; wash up nicely in dishwasher.,These forks are great for my 10 month old daug...,2015-08-31
4,US,46263340,R14I3ZG5E6S7YM,B00KM60D3Q,305813185,Summer Infant Ultra Sight Pan/Scan/Zoom Video ...,Baby,5,0,0,N,Y,Love it!,I wanted something for piece of mind with my l...,2015-08-31


In [346]:
data.describe()

,customer_id,product_parent,star_rating,helpful_votes,total_votes
count,5.000000e+03,5.000000e+03,5000.000000,5000.000000,5000.000000
mean,2.459909e+07,4.894207e+08,4.210200,0.641200,0.834200
std,1.572903e+07,2.849076e+08,1.282788,7.071921,7.367331
min,1.013400e+04,2.341800e+04,1.000000,0.000000,0.000000
25%,1.193007e+07,2.438054e+08,4.000000,0.000000,0.000000
50%,2.280712e+07,4.852615e+08,5.000000,0.000000,0.000000
75%,3.867224e+07,7.304523e+08,5.000000,0.000000,0.000000
max,5.307619e+07,9.992091e+08,5.000000,378.000000,383.000000


In [347]:
data.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

Data preprocessing: text data

In [10]:
text=pd.DataFrame(data['review_headline']+[' ']+data['review_body'])

data['reviews']=text

In [11]:
data.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,reviews
0,US,9970739,R8EWA1OFT84NX,B00GSP5D94,329991347,Summer Infant SwaddleMe Adjustable Infant Wrap...,Baby,5,0,0,N,Y,Great swaddled blankets,Loved these swaddle blankets and so did my dau...,2015-08-31,Great swaddled blankets Loved these swaddle bl...
1,US,23538442,R2JWY4YRQD4FOP,B00YYDDZGU,646108902,Pacifier Clip Girl (3 Pack) Ziggy Baby 2-Sided...,Baby,5,0,0,N,N,Too cute and really nice,These are adorable pacifier clips. SavvyBaby h...,2015-08-31,Too cute and really nice These are adorable pa...
2,US,8273344,RL5ESX231LZ0B,B00BUBNZC8,642922361,Udder Covers - Breast Feeding Nursing Cover,Baby,5,0,0,N,Y,Five Stars,Great gift,2015-08-31,Five Stars Great gift
3,US,24557753,RRMS9ZWJ2KD08,B00AWLZFTS,494272733,Gerber Graduates Fun Pack Utensils,Baby,5,0,0,N,Y,Cute; wash up nicely in dishwasher.,These forks are great for my 10 month old daug...,2015-08-31,Cute; wash up nicely in dishwasher. These fork...
4,US,46263340,R14I3ZG5E6S7YM,B00KM60D3Q,305813185,Summer Infant Ultra Sight Pan/Scan/Zoom Video ...,Baby,5,0,0,N,Y,Love it!,I wanted something for piece of mind with my l...,2015-08-31,Love it! I wanted something for piece of mind ...


In [12]:
data.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date', 'reviews'],
      dtype='object')

In [13]:
data=data.drop(['marketplace','review_id','product_parent','product_category',
                'customer_id','product_id','product_title','verified_purchase',
                'review_headline', 'review_body','review_date'],axis=1)

In [199]:
df=data.copy()

df.head(5)

,star_rating,helpful_votes,total_votes,vine,reviews
0,5,0,0,N,Great swaddled blankets Loved these swaddle bl...
1,5,0,0,N,Too cute and really nice These are adorable pa...
2,5,0,0,N,Five Stars Great gift
3,5,0,0,N,Cute; wash up nicely in dishwasher. These fork...
4,5,0,0,N,Love it! I wanted something for piece of mind ...


In [200]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext


In [201]:

spark = sparknlp.start(spark32 = True)

In [202]:
from pyspark.sql import SparkSession, SQLContext

In [203]:
sc=SparkContext.getOrCreate()

In [204]:

ss=SparkSession(sc)
from pyspark.sql.types import StringType, StructField, StructType,IntegerType,ArrayType

schema= StructType(
      [StructField('star_rating',IntegerType(),True),
      StructField('helpful_votes',IntegerType(),True),
      StructField('total_votes',IntegerType(),True),
      StructField('vine',StringType(),True),
      StructField('reviews',StringType(),True)])

spark_df= ss.createDataFrame(df, schema= schema)



In [205]:
spark_df.show(5)

+-----------+-------------+-----------+----+--------------------+
|star_rating|helpful_votes|total_votes|vine|             reviews|
+-----------+-------------+-----------+----+--------------------+
|          5|            0|          0|   N|Great swaddled bl...|
|          5|            0|          0|   N|Too cute and real...|
|          5|            0|          0|   N|Five Stars Great ...|
|          5|            0|          0|   N|Cute; wash up nic...|
|          5|            0|          0|   N|Love it! I wanted...|
+-----------+-------------+-----------+----+--------------------+
only showing top 5 rows



In [206]:
### preprocessing text data
from pyspark.ml.feature import SQLTransformer, RegexTokenizer, StopWordsRemover, CountVectorizer, Imputer, IDF
from pyspark.ml.feature import StringIndexer, VectorAssembler


stopwords=StopWordsRemover.loadDefaultStopWords('english')

In [207]:
### 1. all words to lowercase

from pyspark.sql.functions import lower, col

import pyspark.sql.functions as f

spark_df=spark_df.select('*', f.lower("reviews"))
spark_df=spark_df.drop('reviews')
spark_df=spark_df.withColumnRenamed('lower(reviews)','reviews') 
spark_df.show(5)

+-----------+-------------+-----------+----+--------------------+
|star_rating|helpful_votes|total_votes|vine|             reviews|
+-----------+-------------+-----------+----+--------------------+
|          5|            0|          0|   N|great swaddled bl...|
|          5|            0|          0|   N|too cute and real...|
|          5|            0|          0|   N|five stars great ...|
|          5|            0|          0|   N|cute; wash up nic...|
|          5|            0|          0|   N|love it! i wanted...|
+-----------+-------------+-----------+----+--------------------+
only showing top 5 rows



In [208]:

### 2. remove punctuations 

from pyspark.sql.functions import regexp_replace, trim, lit, udf

In [209]:
def removePunctuation(column):
    return trim(regexp_replace(column,'\\p{Punct}','')).alias('sentence')

In [210]:
a=spark_df.select(removePunctuation(col('reviews')))

In [211]:
a.show(5)

+--------------------+
|            sentence|
+--------------------+
|great swaddled bl...|
|too cute and real...|
|five stars great ...|
|cute wash up nice...|
|love it i wanted ...|
+--------------------+
only showing top 5 rows



In [212]:
df1 = spark_df.withColumn("id_col", f.monotonically_increasing_id())
df2 = a.withColumn("id_col", f.monotonically_increasing_id())

In [213]:
df3 = df1.join(df2, on="id_col", how="inner")
# drop the id_col column after this
spark_df = df3.drop("id_col",'reviews')

spark_df.show(5)

+-----------+-------------+-----------+----+--------------------+
|star_rating|helpful_votes|total_votes|vine|            sentence|
+-----------+-------------+-----------+----+--------------------+
|          3|            1|          1|   N|a bit messy i ord...|
|          5|            0|          0|   N|it has great room...|
|          5|            0|          0|   N|five stars these ...|
|          3|            1|          4|   N|and the mulberry ...|
|          5|            0|          0|   N|great seat for a ...|
+-----------+-------------+-----------+----+--------------------+
only showing top 5 rows



In [214]:
### 3. tokenization 

from pyspark.ml.feature import SQLTransformer, RegexTokenizer, StopWordsRemover, CountVectorizer, Imputer, IDF
from pyspark.ml.feature import StringIndexer, VectorAssembler


stopwords=StopWordsRemover.loadDefaultStopWords('english')




In [215]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [216]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
spark_df = tokenizer.transform(spark_df)

In [217]:
spark_df=spark_df.drop('sentence')

In [218]:
### 4. remove stopwords

remover=StopWordsRemover(inputCol='words',outputCol='word_removed')

In [219]:
spark_df=remover.transform(spark_df)

In [220]:
spark_df=spark_df.drop('words')

spark_df.show(5)

+-----------+-------------+-----------+----+--------------------+
|star_rating|helpful_votes|total_votes|vine|        word_removed|
+-----------+-------------+-----------+----+--------------------+
|          3|            1|          1|   N|[bit, messy, orde...|
|          5|            0|          0|   N|[great, room, bot...|
|          5|            0|          0|   N|[five, stars, cut...|
|          3|            1|          4|   N|[mulberry, color,...|
|          5|            0|          0|   N|[great, seat, gre...|
+-----------+-------------+-----------+----+--------------------+
only showing top 5 rows



In [282]:
### 5. Count Vectorizer: aim to help convert a collection of text documents to vectors of token counts

from pyspark.ml import Pipeline

cv= CountVectorizer(inputCol= 'word_removed', outputCol= 'rawfeatures')

model=cv.fit(spark_df)
vectorized_df=model.transform(spark_df)

vectorized_df.show(4)

+-----------+-------------+-----------+----+--------------------+--------------------+
|star_rating|helpful_votes|total_votes|vine|        word_removed|         rawfeatures|
+-----------+-------------+-----------+----+--------------------+--------------------+
|          3|            1|          1|   N|[bit, messy, orde...|(10376,[0,49,59,6...|
|          5|            0|          0|   N|[great, room, bot...|(10376,[1,4,8,11,...|
|          5|            0|          0|   N|[five, stars, cut...|(10376,[2,5,23,23...|
|          3|            1|          4|   N|[mulberry, color,...|(10376,[12,24,34,...|
+-----------+-------------+-----------+----+--------------------+--------------------+
only showing top 4 rows



In [256]:
# idf = IDF(inputCol="raw_features", outputCol="features")

# pipeline = Pipeline(stages=[cv, idf])

In [257]:
### 6. Compute Term frequency-inverse document frequency text

# idfModel = pipeline.fit(spark_df)


In [260]:
# total_counts = idfModel.transform(spark_df)\
#                     .select('raw_features').rdd\
#                     .map(lambda row: row['raw_features'].toArray())\
#                     .reduce(lambda x,y: [x[i]+y[i] for i in range(len(y))])



In [283]:
##### 6. Compute Term frequency-inverse document frequency text

from pyspark.ml.feature import HashingTF


hashingTF = HashingTF(inputCol="word_removed", outputCol="rawFeatures", numFeatures=10000)
featurizedData = hashingTF.transform(vectorized_df)



idf= IDF(inputCol= 'rawFeatures', outputCol= 'word_idf')

In [284]:
idfModel = idf.fit(featurizedData)
rescaled_df = idfModel.transform(featurizedData)

In [226]:

rescaled_df.show(4)

+-----------+-------------+-----------+----+--------------------+--------------------+--------------------+--------------------+
|star_rating|helpful_votes|total_votes|vine|        word_removed|          word_count|         rawFeatures|            word_idf|
+-----------+-------------+-----------+----+--------------------+--------------------+--------------------+--------------------+
|          3|            1|          1|   N|[bit, messy, orde...|(10376,[0,49,59,6...|(20,[0,4,6,7,9,10...|(20,[0,4,6,7,9,10...|
|          5|            0|          0|   N|[great, room, bot...|(10376,[1,4,8,11,...|(20,[0,2,3,5,6,7,...|(20,[0,2,3,5,6,7,...|
|          5|            0|          0|   N|[five, stars, cut...|(10376,[2,5,23,23...|(20,[2,9,14],[2.0...|(20,[2,9,14],[0.8...|
|          3|            1|          4|   N|[mulberry, color,...|(10376,[12,24,34,...|(20,[1,2,4,5,8,9,...|(20,[1,2,4,5,8,9,...|
+-----------+-------------+-----------+----+--------------------+--------------------+-----------

In [227]:
### 

clean_df=rescaled_df.drop('word_removed','word_count','rawFeatures')



In [228]:
### convert binary category to numeric 

from pyspark.ml.feature import StringIndexer

In [229]:

indexer = StringIndexer(inputCol="vine", outputCol="vineIndex")
clean_df = indexer.fit(clean_df).transform(clean_df)


In [230]:
clean_df.show(3)
clean_df=clean_df.drop('vine')

+-----------+-------------+-----------+----+--------------------+---------+
|star_rating|helpful_votes|total_votes|vine|            word_idf|vineIndex|
+-----------+-------------+-----------+----+--------------------+---------+
|          3|            1|          1|   N|(20,[0,4,6,7,9,10...|      0.0|
|          5|            0|          0|   N|(20,[0,2,3,5,6,7,...|      0.0|
|          5|            0|          0|   N|(20,[2,9,14],[0.8...|      0.0|
+-----------+-------------+-----------+----+--------------------+---------+
only showing top 3 rows



In [231]:
### create target variable: Good or Bad product 
from pyspark.ml.feature import IndexToString
from pyspark.sql.functions import when





In [232]:
clean_df=clean_df.withColumn('quality', f.when(clean_df['star_rating']==1,'N').otherwise('Y'))

clean_df=clean_df.drop('star_rating')
clean_df.show(3)

+-------------+-----------+--------------------+---------+-------+
|helpful_votes|total_votes|            word_idf|vineIndex|quality|
+-------------+-----------+--------------------+---------+-------+
|            1|          1|(20,[0,4,6,7,9,10...|      0.0|      Y|
|            0|          0|(20,[0,2,3,5,6,7,...|      0.0|      Y|
|            0|          0|(20,[2,9,14],[0.8...|      0.0|      Y|
+-------------+-----------+--------------------+---------+-------+
only showing top 3 rows



In [233]:
######

from pyspark.sql.types import  DoubleType

indices_udf = udf(lambda vector: vector.indices.tolist(), ArrayType(IntegerType()))
values_udf = udf(lambda vector: vector.toArray().tolist(), ArrayType(DoubleType()))

clean_df=clean_df.withColumn('indices', indices_udf(f.col('word_idf')))\
                 .withColumn('values', values_udf(f.col('word_idf')))
                

In [234]:
clean_df.show(3)

+-------------+-----------+--------------------+---------+-------+--------------------+--------------------+
|helpful_votes|total_votes|            word_idf|vineIndex|quality|             indices|              values|
+-------------+-----------+--------------------+---------+-------+--------------------+--------------------+
|            1|          1|(20,[0,4,6,7,9,10...|      0.0|      Y|[0, 4, 6, 7, 9, 1...|[0.60131507511611...|
|            0|          0|(20,[0,2,3,5,6,7,...|      0.0|      Y|[0, 2, 3, 5, 6, 7...|[0.60131507511611...|
|            0|          0|(20,[2,9,14],[0.8...|      0.0|      Y|          [2, 9, 14]|[0.0, 0.0, 0.8241...|
+-------------+-----------+--------------------+---------+-------+--------------------+--------------------+
only showing top 3 rows



In [285]:
#### final check point
clean_df=clean_df.drop('word_idf','indices')


+-------------+-----------+---------+-------+--------------------+
|helpful_votes|total_votes|vineIndex|quality|              values|
+-------------+-----------+---------+-------+--------------------+
|            1|          1|      0.0|      Y|[0.60131507511611...|
|            0|          0|      0.0|      Y|[0.60131507511611...|
|            0|          0|      0.0|      Y|[0.0, 0.0, 0.8241...|
+-------------+-----------+---------+-------+--------------------+
only showing top 3 rows



In [238]:
data_preprocessed=clean_df.toPandas()

In [239]:
data_preprocessed.head(3)

,helpful_votes,total_votes,vineIndex,quality,values
0,1,1,0.0,Y,"[0.6013150751161178, 0.0, 0.0, 0.0, 0.83767953..."
1,0,0,0.0,Y,"[0.6013150751161178, 0.0, 1.2362569677412605, ..."
2,0,0,0.0,Y,"[0.0, 0.0, 0.8241713118275069, 0.0, 0.0, 0.0, ..."


In [316]:

##### deep neural networks, NB, SVM 

data_preprocessed['most_frequency']=np.nan

for i in range(len(data_preprocessed)):
    h= max(data_preprocessed['values'][i])
    data_preprocessed.at[i,'most_frequency']=h

In [318]:
data_preprocessed=data_preprocessed.drop('values',axis=1)

In [319]:
#### random split data 

from sklearn.model_selection import train_test_split

target=data_preprocessed['quality'].astype('category').cat.codes

inputs=data_preprocessed[['helpful_votes','total_votes','vineIndex','most_frequency']]

x_train,x_test,y_train,y_test=train_test_split(inputs,target,test_size=0.2,random_state=20)


In [320]:
#### Deep Neural network 

# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 2 (classes)
layers = 50
input_size=2
output_size=2

In [321]:
from tensorflow import keras

cnn_model=keras.Sequential([
                            keras.layers.Dense(layers, activation='relu'),
                            keras.layers.Dense(layers, activation='relu'),
                            keras.layers.Dense(output_size, activation='softmax')
])


In [322]:
cnn_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [323]:
cnn_model.fit(x_train,y_train,epochs=100,batch_size=100, validation_data=(x_test,y_test))

Epoch 1/100
40/40 [==============================] - 1s 8ms/step - loss: 0.4451 - accuracy: 0.9122 - val_loss: 0.3701 - val_accuracy: 0.9330
Epoch 2/100
40/40 [==============================] - 0s 3ms/step - loss: 0.3683 - accuracy: 0.9110 - val_loss: 0.3413 - val_accuracy: 0.9330
Epoch 3/100
40/40 [==============================] - 0s 3ms/step - loss: 0.3362 - accuracy: 0.9120 - val_loss: 0.3158 - val_accuracy: 0.9320
Epoch 4/100
40/40 [==============================] - 0s 3ms/step - loss: 0.3160 - accuracy: 0.9118 - val_loss: 0.2956 - val_accuracy: 0.9320
Epoch 5/100
40/40 [==============================] - 0s 3ms/step - loss: 0.3054 - accuracy: 0.9110 - val_loss: 0.2763 - val_accuracy: 0.9310
Epoch 6/100
40/40 [==============================] - 0s 3ms/step - loss: 0.3052 - accuracy: 0.9120 - val_loss: 0.2663 - val_accuracy: 0.9310
Epoch 7/100
40/40 [==============================] - 0s 3ms/step - loss: 0.2917 - accuracy: 0.9125 - val_loss: 0.2599 - val_accuracy: 0.9310
Epoch 8/100
4

In [324]:

## test the model 

test_accuracy=cnn_model.evaluate(x_test,y_test)

32/32 [==============================] - 0s 3ms/step - loss: 0.2431 - accuracy: 0.9360


In [330]:
#### Naive Bayes

from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score

In [331]:
gnb_model=GaussianNB()

In [335]:
y_pred=gnb_model.fit(x_train, y_train).predict(x_test)

In [336]:
accuracy_score(y_test, y_pred)

0.082

In [338]:
### Support Vector Machine 

from sklearn.svm import SVC

In [339]:
svm=SVC()

In [340]:
svm_model=svm.fit(x_train,y_train)

In [341]:
y_pred=svm_model.predict(x_test)

In [342]:
accuracy_score(y_test, y_pred)

0.933